## Background

Simplified way to interact with some selected Maxar Open Data STAC collections.

Assumes you've run the repo installation instructions and have the docker services running.

## Import libraries

In [1]:
from datetime import datetime

import geopandas as gpd

from eo_maxar.util import MaxarCollection, get_collections

View what collections are available.

In [2]:
get_collections()

['MAXAR_afghanistan_earthquake22',
 'MAXAR_BayofBengal_Cyclone_Mocha_May_23',
 'MAXAR_cyclone_emnati22',
 'MAXAR_Emilia_Romagna_Italy_flooding_may23',
 'MAXAR_Gambia_flooding_8_11_2022',
 'MAXAR_ghana_explosion22',
 'MAXAR_Hurricane_Fiona_9_19_2022',
 'MAXAR_Hurricane_Ian_9_26_2022',
 'MAXAR_Hurricane_Idalia_Florida_Aug23',
 'MAXAR_India_Floods_Oct_2023']

## Turkey & Syria earthquake

We'll get started with the 2023 Turkey & Syria earthquake to visualise collections of Cloud-Optimised GeoTIFFs pre and post event.

The `MaxarCollection` is a pydantic base model and comes with various methods to make it simpler to interact with a collection.

In [3]:
collection_id = "MAXAR_Kahramanmaras_turkey_earthquake_23"

turkey_earthquake = MaxarCollection(collection_id=collection_id)

event_date = datetime(2023, 2, 6, hour=0, minute=0)  # noqa: DTZ001

We can use `get_collection_info()` to return some selected information from the collection.

In [4]:
for attribute, value in vars(turkey_earthquake.get_collection_info()).items():
    print(f"{attribute}: {value}")

id: MAXAR_Kahramanmaras_turkey_earthquake_23
title: Turkey and Syria Earthquake 2023
description: Maxar OpenData | A devastating magnitude 7.8 earthquake struck the Turkish province of Kahramanmaras, approximately 23 kilometers east of Nurdagi in the Gaziantep province near the Syrian border, at 4:17 a.m. local time on Monday, February 6, 2023, followed by a 7.5 magnitude aftershock nine hours later. More than 6,000 people have died in Turkey and Syria, and tens of thousands of people have been injured. Those numbers are expected to increase as search and rescue activities continue. At least 13 million people in the region have been impacted by the earthquake and aftershock. Turkey's president declared a three-month state of emergency in the 10 provinces hardest hit by the earthquake.
extent: {'spatial': {'bbox': [[35.302597, 35.875122, 40.310497, 38.47292570695286], [37.2976, 36.98959965805714, 37.47444448907068, 37.015901889979396], [37.29774464331677, 36.9896650792383, 37.4575240343

### Collection map

We can quickly map the extents of all items within a collection.

The blue boxes specify the extent of each item, whereas the dashed black line is the extent of the entire collection.

In [5]:
m = turkey_earthquake.collection_bbox_map()
m

Map(center=[37.17402385347643, 37.806546999999995], controls=(ZoomControl(options=['position', 'zoom_in_text',…

We can return every single item from the collection.

These contain important information elating to the item, including:
- `id`: each unique ID to identify specific items
- `bbox`: bounding box that defines the geographical area the item covers.
- `datetime`: indicates date and time item was collected.
- `assets`: contains information on how to access specific variables of the item (e.g. visual, multispectral, panchromatic etc.)
- `properties`: metadata of the item such as ground sampling distance, UTM zone, projection system, cloud cover.
- `geometry`: geographical coordinates to define the shape (`Polygon`) of the item.

You can find more on the specification: https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md

In [7]:
items = turkey_earthquake.get_collection_items()

### Pre-Post event items
Once we've got the collection items, we can split them into a pre and post event collection.

In [ ]:
m = turkey_earthquake.pre_post_map(items, event_date)
m

### Single COG image
In our docker services, we have a raster API endpoint connected to the PgSTAC database. The service is built using [titiler-pgstac](http://github.com/stac-utils/titiler-pgstac) and can be used to visualize individual `Item` or `Mosaics` (multiple items).

Raster endpoint: `http://localhost:8082`

Each item has 4 assets and 3 of which are `Cloud-Optimised GeoTIFF` types which is what we're interested in in terms of earth observation data.

Below we'll use the `single_cog_map()` method to view a single item using the `visual` asset - this is the default asset we use.

In [ ]:
turkey_earthquake.single_cog_map(items, items[0]["id"])

But you can also view any of the COG files, below using the panchromatic image.

In [ ]:
turkey_earthquake.single_cog_map(items, items[0]["id"], asset="pan_analytic")

But the majority of the time we'll want the `visual` asset.

### Mosaic map
Although we can use to `titiler-pgstac` to visualise individual item assets, the raster APIs real benefit is creating a virtual mosaic dynamically and merging multiple items on the fly.

These are created by making `httpx.post` request to the raster API to find all items that match a CQL filter, and are then sorted by the `tile:clouds_percent` and `direction` properties.

We can then use this search request to create the virtual mosaic raster on the fly.

Below we'll create a mosaic for Antakya in the Hatay region before the earthquake.

In [9]:
bounds = [36.129570, 36.180701, 36.194458, 36.224058]

turkey_earthquake.mosaic_map(
    items=items,
    bbox=bounds,
    event_date=event_date,
    pre_event=True,
)

Map(center=[36.2023795, 36.162014], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

And below, we can view it after the earthquake.

In [ ]:
turkey_earthquake.mosaic_map(
    items=items,
    bbox=bounds,
    event_date=event_date,
    pre_event=False,
)

### Mosaic split map
We can also create a split map for a before and after event using the virtual mosaics, making it simpler visualise changes.

Can also provide custom map keyword arguments.

In [10]:
turkey_earthquake.mosaic_split_map(
    items=items,
    bbox=bounds,
    event_date=event_date,
    map_kwargs={
        "center": ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2),
        "zoom": 16,
        "layout": {"height": "650px"},
    },
)

Map(center=[36.2023795, 36.162014], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## Ghana explosion

We can use the UI frontend to explore other collections, such as the [Ghana explosion](http://localhost:8085/collections/MAXAR_ghana_explosion22).

On 22/01/2022 a motorbike collied with a truck carrying 10 tons of explosives causing a blast which levelled the town of Apiate, located in western Ghana.

We can use the same approach to quickly visalise the impact.


In [ ]:
collection_id = "MAXAR_ghana_explosion22"

ghana_explosion = MaxarCollection(collection_id=collection_id)

for attribute, value in vars(ghana_explosion.get_collection_info()).items():
    print(f"{attribute}: {value}")

We can use tools from `geopandas` to geocode areas and make it a little easier for constructing the map requirements.

In [14]:
gdf = gpd.tools.geocode("Apiate, Ghana")

bounds = list(
    map(
        float, gdf.to_crs(gdf.estimate_utm_crs()).buffer(1000).to_crs(4326).total_bounds
    )
)

event_date = datetime(2022, 1, 19, hour=0, minute=0)  # noqa: DTZ001

items = ghana_explosion.get_collection_items()

In [ ]:
ghana_explosion.mosaic_split_map(
    items=items,
    bbox=bounds,
    event_date=event_date,
    map_kwargs={
        "center": ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2),
        "zoom": 18,
        "layout": {"height": "700px"}
    },
)

## Hurricane Ian

Another example from the [Hurricane Ian collection](http://localhost:8085/collections/MAXAR_Hurricane_Ian_9_26_2022).

Hurricane Ian made landfall as a Category 4 storm near Cay Costa, Florida, on Wednesday, September 28, 2022, with winds of 150 mph and with record storm surge flooding as high as 12 feet in some coastal areas.

We'll have a quick look at the before and after on Sanibel Island, Florida.

In [ ]:
collection_id = "MAXAR_Hurricane_Ian_9_26_2022"

hurricane_ian = MaxarCollection(collection_id=collection_id)

for attribute, value in vars(hurricane_ian.get_collection_info()).items():
    print(f"{attribute}: {value}")

In [22]:
gdf = gpd.tools.geocode("Sanibel Island, Florida")

bounds = list(
    map(
        float, gdf.to_crs(gdf.estimate_utm_crs()).buffer(5000).to_crs(4326).total_bounds
    )
)

event_date = datetime(2022, 9, 28, hour=0, minute=0)  # noqa: DTZ001

In [6]:
items = hurricane_ian.get_collection_items()

Post-event mosaic.

In [ ]:
hurricane_ian.mosaic_map(
    items=items,
    bbox=bounds,
    event_date=event_date,
    pre_event=False,
    map_kwargs={
        "center": ((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2),
        "zoom": 12,
        "layout": {"height": "650px"}
    },
)